In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e5/sample_submission.csv
/kaggle/input/playground-series-s5e5/train.csv
/kaggle/input/playground-series-s5e5/test.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import joblib

# from src.data_preprocessing import clean_data
# from src.feature_engineering import add_cross_features

In [3]:
data_folder = "/kaggle/input/playground-series-s5e5"

In [4]:
df_train = pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv')
df_test  = pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv')

In [5]:
df_train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [6]:
df_test.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8
1,750001,male,26,200.0,97.0,20.0,101.0,40.5
2,750002,female,29,188.0,85.0,16.0,102.0,40.4
3,750003,female,39,172.0,73.0,20.0,107.0,40.6
4,750004,female,30,173.0,67.0,16.0,94.0,40.5


extract feature

In [7]:
# สร้างคอลัมน์ HRmax ให้กับ df_train และ df_test
df_train['HRmax'] = df_train['Age'].apply(lambda age: 220 - age)
df_test['HRmax'] = df_test['Age'].apply(lambda age: 220 - age)

# กำหนดค่า RHR คงที่
RHR = 65

# คำนวณ VO2MAX สำหรับ train และ test
df_train["VO2MAX"] = 15.3 * (df_train["HRmax"] / RHR)
df_test["VO2MAX"] = 15.3 * (df_test["HRmax"] / RHR)


df_train["METmax"] = df_train["VO2MAX"] / 3.5
df_test["METmax"] = df_test["VO2MAX"] / 3.5

df_train['MET'] = ((df_train['Heart_Rate'] - RHR) / (df_train['HRmax'] - RHR)) * (df_train['METmax'] - 1) + 1
df_test['MET'] = ((df_test['Heart_Rate'] - RHR) / (df_test['HRmax'] - RHR)) * (df_test['METmax'] - 1) + 1

df_train['calories_cal'] = (df_train['MET'] * df_train['Weight'] * df_train['Duration']) / 60
df_test['calories_cal'] = (df_test['MET'] * df_test['Weight'] * df_test['Duration']) / 60


In [8]:
df_train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,HRmax,VO2MAX,METmax,MET,calories_cal
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0,184,43.310769,12.374505,4.441027,157.804488
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0,156,36.720000,10.491429,3.086028,24.688226
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0,169,39.780000,11.365714,2.893736,21.606564
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0,200,47.076923,13.450549,4.689052,175.839438
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0,182,42.840000,12.240000,4.554530,115.760969


In [9]:
df_test.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,HRmax,VO2MAX,METmax,MET,calories_cal
0,750000,male,45,177.0,81.0,7.0,87.0,39.8,175,41.192308,11.769231,3.153846,29.803846
1,750001,male,26,200.0,97.0,20.0,101.0,40.5,194,45.664615,13.047033,4.361963,141.036794
2,750002,female,29,188.0,85.0,16.0,102.0,40.4,191,44.958462,12.845275,4.478374,101.509818
3,750003,female,39,172.0,73.0,20.0,107.0,40.6,181,42.604615,12.172747,5.045305,122.769089
4,750004,female,30,173.0,67.0,16.0,94.0,40.5,190,44.723077,12.778022,3.732501,66.687353


In [10]:
# Select numerical columns
numerical_cols = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']